In [1]:
from bs4 import BeautifulSoup
import requests
#import urllib2
import re


In [2]:
#enter artists page here
base = 'https://michellezauner.bandcamp.com'

In [3]:
def get_albums(base):
    """
    helper function
    input: artist page
    output: link to each album by artist on bandcamp
    """
    page_link = base + '/music'
    page_response = requests.get(page_link)
    page_content = BeautifulSoup(page_response.content, "lxml")
    links = []
    #regex search for links with album
    for link in page_content.find_all('a', href=re.compile('^/album/*')):
        links.append(link.get('href'))
    #add base to relative links
    links = [base + link for link in links]
    return links
    
    
def get_tracks(link, track_links = []):
    """
    helper function
    input: link from list output by get_albums, list of tracks
    output: appended tracks from album to list
    
    """
    page_link = link
    page_response = requests.get(page_link)
    page_content = BeautifulSoup(page_response.content, "lxml")
    for link in page_content.find_all('a', href=re.compile('^/track/*')):
        track_links.append(link.get('href'))
    return track_links

def all_tracks(base):
    """
    user function
    input: base link
    output: link to each track by artist
    """
    links = get_albums(base)
    track_links = []
    for link in links:
        track_links = get_tracks(link, track_links)
    return track_links

In [4]:
tracks = all_tracks('https://michellezauner.bandcamp.com')

In [5]:
import pandas as pd

In [6]:
#make a dataframe of all the tracks
tracks = pd.DataFrame({'tracks':tracks})

In [7]:
#filter out the redundant tracks (lyrics or purchase links)
tracks = tracks[~tracks['tracks'].str.contains("#|\?")]

In [8]:
def get_lyrics(track, base):
    """
    input: relative link to track
    base: bandcamp base page
    """
    page_link = base + track
    page_response = requests.get(page_link)
    page_content = BeautifulSoup(page_response.content, "lxml")
    # bandcamp has a nice section for the lyrics so it is easy to get
    lyrics = page_content.find('div', class_="lyricsText").get_text()
    return lyrics

In [ ]:
#store as a dict to easily make into a dataframe later
lyric_dict = {}
for track in tracks['tracks']:
    print(track)
    try:
        #call lyric function
        lyrics = get_lyrics(track = track, base = 'https://michellezauner.bandcamp.com')
        #split to get track name
        lyric_dict[re.split("/", track)[-1]] = lyrics
    except:
        pass
    

/track/diving-woman
/track/road-head-2
/track/machinist
/track/planetary-ambience
/track/soft-sounds-from-another-planet
/track/boyish
/track/12-steps
/track/jimmy-fallon-big-2
/track/the-body-is-a-blade
/track/till-death
/track/this-house
/track/here-come-the-tubular-bells
/track/in-heaven-2
/track/the-woman-that-loves-you-3
/track/rugged-country-2
/track/everybody-wants-to-love-you-2
/track/psychopomp-2
/track/jane-cum-3
/track/heft-3
/track/moon-on-the-bath-2
/track/triple-7-3
/track/intro
/track/intro
/track/the-woman-that-loves-you
/track/jane-cum
/track/road-head
/track/beating-off
/track/bread
/track/triple-7
/track/jimmy-fallon-big
/track/day-1
/track/day-2
/track/day-2
/track/day-3
/track/day-4
/track/day-5
/track/day-6
/track/day-7
/track/day-8
/track/day-9
/track/day-10
/track/day-11
/track/day-12
/track/day-13
/track/day-14
/track/day-15
/track/day-15
/track/day-16
/track/day-17
/track/day-18


In [101]:
df = pd.DataFrame.from_dict(lyric_dict, orient='index').reset_index()
df = df.rename(index=str, columns={"index": "song", 0: "lyrics"})

In [116]:
#test
#for lyric in df['lyrics'].values.tolist():
#    print(lyric)

I want to be a woman of regimen
A bride in her home state
A diving woman of Jeju-do

I want it all
I want it

The men have gone and left again
And no one’s shocked or blames them
Another day is shot, my friend
I’m picking up the pieces
That’s the third dead dog I’ve seen on this highway
These treads are wearing thin
When I get back there, baby
I’m gonna make you a home

You’ll have it all
I’ll have it
You gave road head on a turnpike exit
Going home, going home
Last ditch desperate, like a makeshift siphon
Pump and run, pump and run
Pump and run, pump and run

Run!

“Dream on, baby,” were his last words to me
“Dream on, baby.”
So dreaming baby took that corkscrewed highway
Lightless miles of big rigs
Lightless miles, miles and miles

Home
[Woman] 
I don’t know how it happened
Was it always this way and I just couldn’t see it?
Heart burning hot enough for the both of us
I never realized how much you were holding back
All the times I felt so plugged in, you were tuning out
A muted channe

In [115]:
# remove title if title is in lyrics
df['lyrics'] = df['lyrics'].map(lambda a: re.sub('^[A-Z].*:\r\n\n','', a))

In [117]:
save to pickle
df.to_pickle('jbrekkie_lyrics.pkl')